# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,3369157,Cape Town,20.0,190.00,ZA,29131.0,288.25,290.93,292.04,1016.0,77.0,NaN,NaN,-51.968991,-3.848592
1,1,6165406,Thompson,20.0,260.00,CA,24615.0,240.15,246.15,246.15,1023.0,75.0,NaN,NaN,59.750385,-97.258161
2,2,3395981,Maceió,20.0,120.00,BR,29915.0,301.51,299.15,299.15,1013.0,74.0,NaN,NaN,NaN,NaN
3,3,3374210,São Filipe,61.0,56.00,CV,29629.0,295.63,296.29,296.29,1014.0,74.0,1014.0,1009.0,NaN,NaN
4,4,3874787,Punta Arenas,40.0,260.00,CL,2794.0,267.36,278.71,280.15,997.0,65.0,NaN,NaN,-79.100128,-100.457137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,572,2015852,Svetlaya,86.0,297.00,RU,25703.0,248.77,257.03,257.03,1009.0,81.0,1009.0,1008.0,39.219776,44.174836
573,573,3105522,Bereda,98.0,5.81,ES,28036.0,276.55,279.82,281.48,1009.0,99.0,NaN,NaN,70.242092,164.509014
574,574,3449053,São Gotardo,54.0,292.00,BR,29304.0,294.92,293.04,293.04,1012.0,84.0,1012.0,898.0,NaN,NaN
575,575,2352250,Afikpo,90.0,205.00,NG,29826.0,301.48,298.26,298.26,1010.0,85.0,1010.0,998.0,-74.114852,24.920317


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [4]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 4)

# fig.add_layer(heat_layer)

# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Country,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
163,163,3386177,Trairi,75.0,85.0,BR,29977.0,301.88,299.77,299.77,1010.0,77.0,1010.0,1009.0,-72.661170,-1.809743
256,256,2047258,Novyy Urgal,100.0,57.0,RU,2464.0,242.20,246.40,246.40,1020.0,86.0,1020.0,978.0,87.903181,-64.847792
208,208,1712961,Gigmoto,95.0,73.0,PH,30019.0,302.27,300.19,300.19,1012.0,82.0,1012.0,1011.0,42.321149,-120.695972
527,527,2449893,Tessalit,77.0,45.0,ML,29215.0,287.94,292.15,292.15,1013.0,25.0,1013.0,952.0,-81.611943,-120.083448
156,156,2022773,Khandyga,53.0,125.0,RU,22439.0,219.87,224.39,224.39,1035.0,61.0,1035.0,1012.0,-76.487755,4.146060
282,282,2162683,Innisfail,85.0,107.0,AU,30317.0,307.18,303.17,303.17,1008.0,60.0,1008.0,1007.0,-21.210882,25.626960
511,511,2134814,Sola,77.0,244.0,VU,301.0,302.52,301.00,301.00,1007.0,81.0,1007.0,1000.0,49.479118,70.596522
209,209,3839307,Rawson,62.0,136.0,AR,2867.0,283.14,286.70,286.70,1004.0,41.0,1004.0,1003.0,-14.646607,-126.587992
414,414,5975034,High Prairie,100.0,290.0,CA,25715.0,252.21,257.15,257.15,1024.0,91.0,1024.0,947.0,72.938432,-126.588097
519,519,1497210,Novoagansk,100.0,5.0,RU,25956.0,253.35,259.56,259.56,1018.0,95.0,1018.0,1010.0,25.458067,-27.016907


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City"})
hotel_df

,index,City,Country,Latitude,Longitude,Hotel Name
0,163,Trairi,BR,-72.661170,-1.809743,
1,256,Novyy Urgal,RU,87.903181,-64.847792,
2,208,Gigmoto,PH,42.321149,-120.695972,
3,527,Tessalit,ML,-81.611943,-120.083448,
4,156,Khandyga,RU,-76.487755,4.146060,
5,282,Innisfail,AU,-21.210882,25.626960,
6,511,Sola,VU,49.479118,70.596522,
7,209,Rawson,AR,-14.646607,-126.587992,
8,414,High Prairie,CA,72.938432,-126.588097,
9,519,Novoagansk,RU,25.458067,-27.016907,


In [7]:
# #the latitudes and longitudes I saved from citipy were not getting along with the google api at this point, so I decided
#     #to get the place_id from the api and then get google's preferred coordinates from their own native api in order to 
#     #get to the hotel data.

# #get place_id for city
# placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         placeid_params = {
#         "input": row['City'],
#         "inputtype": "textquery",
#         "key": gkey,
#     }
#         ID = requests.get(placeid_url, params=placeid_params).json()
#         ID = (ID["candidates"][0]["place_id"])
#         print(ID)
    

In [8]:
# #get coordinates from place_id
# coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         coordinate_params = {
#         "place_id": ID,
#         "key": gkey,
#     }
# #     assemble url and make API request
#         coords = requests.get(coordinate_url, params=coordinate_params).json()
#         lat = coords['result']['geometry']['location']['lat']
#         lng = coords['result']['geometry']['location']['lng']
#         coords = f'{lat},{lng}'
#         print(coords)

In [12]:
# find the closest restaurant of each type to coordinates


placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    print(f'City = {City}')
    

#     use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords_req = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords_req['result']['geometry']['location']['lat']
    lng = coords_req['result']['geometry']['location']['lng']
    try:
        Country = row["Country"]
        print(f'Country = {Country}')
    except:
        print(f'Country = n/a')

    
    coords = f'{lat},{lng}'
    
    hotel_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    
#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(hotel_url, params=hotel_params).json()
    # extract results
    try:
        results = response['results'][0]['name']
        hotels.append(results)
        print(f'Hotel = {results}')
    except:
        results = {"No hotel found nearby"}
        hotels.append(results)
        print(results)
    
    
    try:
        print(f"The closest hotel to {City} is {results}.")
    except (KeyError, IndexError):
        print(f"Sorry, there are no hotels within 5000 meters of {City}. Maybe try someplace less remote?")
        
    print("------------")
hotel_df["Hotel Name"] = hotels
hotel_df.rename(columns={"Latitude":"Lat", "Longitude":"Lng"}) #not working???
hotel_df


City = Trairi
Country = BR
Retrieving Results for hotels near Trairi:
Hotel = janelas do mar
The closest hotel to Trairi is janelas do mar.
------------
City = Novyy Urgal
Country = RU
Retrieving Results for hotels near Novyy Urgal:
{'No hotel found nearby'}
The closest hotel to Novyy Urgal is {'No hotel found nearby'}.
------------
City = Gigmoto
Country = PH
Retrieving Results for hotels near Gigmoto:
Hotel = NAHULUGAN FALLS
The closest hotel to Gigmoto is NAHULUGAN FALLS.
------------
City = Tessalit
Country = ML
Retrieving Results for hotels near Tessalit:
{'No hotel found nearby'}
The closest hotel to Tessalit is {'No hotel found nearby'}.
------------
City = Khandyga
Country = RU
Retrieving Results for hotels near Khandyga:
Hotel = Prezidentskaya Gostinitsa
The closest hotel to Khandyga is Prezidentskaya Gostinitsa.
------------
City = Innisfail
Country = AU
Retrieving Results for hotels near Innisfail:
Hotel = Days Inn Innisfail
The closest hotel to Innisfail is Days Inn Innisfa

,index,City,Country,Latitude,Longitude,Hotel Name
0,163,Trairi,BR,-72.661170,-1.809743,janelas do mar
1,256,Novyy Urgal,RU,87.903181,-64.847792,{No hotel found nearby}
2,208,Gigmoto,PH,42.321149,-120.695972,NAHULUGAN FALLS
3,527,Tessalit,ML,-81.611943,-120.083448,{No hotel found nearby}
4,156,Khandyga,RU,-76.487755,4.146060,Prezidentskaya Gostinitsa
5,282,Innisfail,AU,-21.210882,25.626960,Days Inn Innisfail
6,511,Sola,VU,49.479118,70.596522,Clayton Plaza Hotel
7,209,Rawson,AR,-14.646607,-126.587992,Shrine Guesthouse
8,414,High Prairie,CA,72.938432,-126.588097,Royal Star Motor Inn
9,519,Novoagansk,RU,25.458067,-27.016907,{No hotel found nearby}


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
